In [1]:
import matplotlib.pyplot as plt  # standard Python plotting library
import numpy as np  # fundamental package for scientific computing, handles arrays and maths
import pandas as pd
import openpyxl
import xlsxwriter
from dateutil import parser
import seaborn
from matplotlib.pyplot import figure
# import the tdt library
import tdt

In [2]:
def fiberphoto_raw_plot(any_t, any_t_end): 
    # declare the figure size
    fig1 = plt.subplots(figsize=(10, 6))

    # plot the line using slices
    plt.plot(_470A_time[any_t: any_t_end], data.streams._470A.data[any_t: any_t_end], color='cornflowerblue', label="Calcium Channel")
    plt.plot(_405A_time[any_t: any_t_end], data.streams._405A.data[any_t: any_t_end], color='deeppink', label="Isobetic Channel")

    # Some matplotlib stuff
    # add an annotation mark to the figure
    # plt.annotate('Point Here',
    #              xy=(0.8,0.002),
    #              xytext=(.88,.003),
    #              arrowprops=dict(arrowstyle='->', color='k')
    #             )
    plt.axline((1427.48041216, 40), (1427.48041216, 54), linewidth=2, color='r')

    # create title, axis labels, and legend
    plt.title('Tail pick up Data', fontsize=16)
    plt.xlabel('Seconds', fontsize=14)
    plt.ylabel('Volts', fontsize=14)
    plt.legend(loc='lower right', 
               bbox_to_anchor=(1.0,1.01)
              )
    plt.autoscale(tight=True)
    plt.show()

F470 = []
F405 = []
N = 10
for i in range(0, len(data.streams._470A.data), N):
    F470.append(np.mean(data.streams._470A.data[i:i+N-1])) # This is the moving window mean
data.streams._470A.data = F470
for i in range(0, len(data.streams._405A.data), N):
    F405.append(np.mean(data.streams._405A.data[i:i+N-1])) # This is the moving window mean
data.streams._405A.data = F470

t = 8
time = np.linspace(1,len(data.streams._470A.data), len(data.streams._470A.data))/data.streams._470A.fs
inds = np.where(time>t)
ind = inds[0][0]
time = time[ind:]

time = time[::N] # go from beginning to end of array in steps on N
time = time[:len(data.streams._470A.data)]

# Detrending and dFF
# Full trace dFF according to Lerner et al. 2015
# https://dx.doi.org/10.1016/j.cell.2015.07.014
# dFF using 405 fit as baseline

x = np.array(data.streams._405A.data)
y = np.array(data.streams._470A.data)
bls = np.polyfit(x, y, 1)
Y_fit_all = np.multiply(bls[0], x) + bls[1]
Y_dF_all = y - Y_fit_all

dFF = np.multiply(100, np.divide(Y_dF_all, Y_fit_all))
std_dFF = np.std(dFF)

x = np.array(data.streams._405A.data)
y = np.array(data.streams._470A.data)
bls = np.polyfit(x, y, 1)
Y_fit_all = np.multiply(bls[0], x) + bls[1]
Y_dF_all = y - Y_fit_all

dFF = np.multiply(100, np.divide(Y_dF_all, Y_fit_all))
std_dFF = np.std(dFF)

In [ ]:
# want to all of the same events from the same file together into one file 
# read the list file first, then read the corresponding fiber photometry data file 
# loop through each file and generate subplot one by one
# in each loop: get the start time and end time based on [Events], state start - 20s, state stop + 20s, 
    # plot the dff from the fiber photometry data file

In [ ]:
# want to plot all of the start to end events in one file, state start - 20s, state stop + 20s,
# point event before 20s and after 40s
# 1. read events log file  & read corresponding data file 
# 2. classify events into different groups based on [behavior]
# 3. two different group, one group of state start, state stop, 
# the other one with point event based on [Event]
# 4. plot all of the graph in one specific behavior with state start and state stop labeled as a row 
# 5. plot all of the graph in one specific behavior with point event as a row 
# save images 

In [ ]:
file_name_list = ["11_FPh3_2R"]

In [ ]:
behavior_log_path = "/Users/sabrinay/Desktop/2022_behavior_data/20220510_fiber_photometry_ucn/20220510_fiber_photometry_data/processed_behavior_scoring/"
data_path = "/Users/sabrinay/Desktop/2022_behavior_data/20220510_fiber_photometry_ucn/20220510_fiber_photometry_data/20220510_fph_refeed_in_oft/"

In [ ]:
BLOCK_PATH = data_path + file_name_list[0] + str("/") 
Log_path = behavior_log_path + file_name_list[0] + ".xlsx"

In [ ]:
BLOCK_PATH

In [ ]:
data = tdt.read_block(BLOCK_PATH)
df = pd.read_excel(Log_path)
df = df.dropna(axis = 0)
num_samples = len(data.streams._470A.data)
_470A_time = np.linspace(1, num_samples, num_samples) / data.streams._470A.fs
_405A_time = np.linspace(1, num_samples, num_samples) / data.streams._405A.fs
dff = []
for i in range (0,len(data.streams._470A.data)):
    dff.append((data.streams._470A.data[i]-data.streams._405A.data[i])/data.streams._405A.data[i])

In [ ]:
df

In [ ]:
# Group by behavior 
d = {}
for i in df['Behavior'].unique():
    d[i] = [{df['Recording time'][j]: df['Event'][j]} for j in df[df['Behavior']==i].index]

In [ ]:
any_t = int((start_time[(2*0 -2)] - 5 )* data.streams._470A.fs)
any_t

In [ ]:
df_point = df[df['Event'].str.contains('point event')]
df_duration = df[~df['Event'].str.contains('point event')]
dfs_point = [x for _, x in df_point.groupby('Behavior')]
dfs_duration = [x for _, x in df_duration.groupby('Behavior')]

In [ ]:
behavior_type = str (dfs_duration[0]['Behavior'].unique())
start_time = dfs_duration[0]['Recording time'].tolist()
lengs = int(len(start_time)/2)
fig, axs = plt.subplots(lengs, figsize=(15,(lengs * 5)))
fig.suptitle(behavior_type + file_name_list[0])
print(lengs)
for j in range (1, lengs): 
    any_t = int((start_time[(2*j -2)] - 5 )* data.streams._470A.fs)
    any_t_end = int((start_time[(2 * j - 1)] + 5) * data.streams._470A.fs)
    axs[j].plot(_470A_time[any_t: any_t_end], dff[any_t: any_t_end], color='cornflowerblue', label="Calcium Channel")
    max_any_t = max(dff[any_t: any_t_end])
    min_any_t = min(dff[any_t: any_t_end])
    axs[j].vlines((start_time[(2*j -2)]), min_any_t, max_any_t, linewidth=2, color='r')
    axs[j].vlines((start_time[(2 * j - 1)]), min_any_t, max_any_t, linewidth=2, color='r')
fig.savefig(behavior_type + file_name_list[0] + '.png')

In [ ]:
n = len(dfs_duration)
for i in range (0, len(dfs_duration)):
    
    behavior_type = str (dfs_duration[i]['Behavior'].unique())
    start_time = dfs_duration[i]['Recording time'].tolist()
    lengs = int(len(start_time)/2)
    fig, axs = plt.subplots(lengs, figsize=(15,(lengs * 5)))
    fig.suptitle(behavior_type + file_name_list[0])
    print(lengs)
    for j in range (1, lengs): 
        any_t = int((start_time[(2*j -2)] - 5 )* data.streams._470A.fs)
        any_t_end = int((start_time[(2 * j - 1)] + 5) * data.streams._470A.fs)
        max_any_t = max(dff[any_t: any_t_end])
        min_any_t = min(dff[any_t: any_t_end])
        axs[j].plot(_470A_time[any_t: any_t_end], dff[any_t: any_t_end], color='cornflowerblue', label="Calcium Channel")
        axs[j].vlines(start_time[2*j -2], min_any_t, max_any_t, linewidth=2, color='r')
        axs[j].vlines(start_time[2 * j - 1], min_any_t, max_any_t, linewidth=2, color='r')
    fig.savefig(behavior_type + file_name_list[0] + '.png')

In [ ]:
n = len(dfs_point)
for i in range (0, len(dfs_point)):
    behavior_type = str (dfs_point[0]['Behavior'].unique())
    start_time = dfs_point[i]['Recording time'].tolist()
    fig, axs = plt.subplots(len(start_time), figsize=(15,100))
    fig.suptitle(behavior_type + file_name_list[0])
    for j in range (0, len(start_time)): 
        any_t = int((start_time[j] - 20 )* data.streams._470A.fs)
        any_t_end = int((start_time[j] + 30) * data.streams._470A.fs)
        axs[j].plot(_470A_time[any_t: any_t_end], dff[any_t: any_t_end], color='cornflowerblue', label="Calcium Channel")
        axs[j].vlines(start_time[j], 0, 0.5, linewidth=2, color='r')
    plt.show()
    plt.savefig(behavior_type + file_name_list[0] + '.png')
    fig.savefig(behavior_type + file_name_list[0] + '.png')

In [ ]:
Behavior_type

In [ ]:
fig3 = plt.subplots(figsize=(10, 6))

plt.plot(_470A_time[tail_t: tail_t_end], dff[tail_t: tail_t_end], color='cornflowerblue', label="Calcium Channel")
# Some matplotlib stuff
# add an annotation mark to the figure
# plt.annotate('Point Here',
#              xy=(0.8,0.002),
#              xytext=(.88,.003),
#              arrowprops=dict(arrowstyle='->', color='k')
#             )
plt.axline((1427.48041216, 0), (1427.48041216, 0.5), linewidth=2, color='r')

# create title, axis labels, and legend
plt.title('Tail pick up Data', fontsize=16)
plt.xlabel('Seconds', fontsize=14)
plt.ylabel(r'$\Delta$F/F')
plt.legend(loc='lower right', 
           bbox_to_anchor=(1.0,1.01)
          )
plt.autoscale(tight=True)
plt.show()

In [ ]:
any_t = int(400 * data.streams._470A.fs)
any_t_end = int(1450 * data.streams._470A.fs)

In [ ]:
fig5 = plt.subplots(figsize=(15, 6))

plt.plot(_470A_time[any_t: any_t_end], dff[any_t: any_t_end], color='cornflowerblue', label="Calcium Channel")
# Some matplotlib stuff
# add an annotation mark to the figure
# plt.annotate('Point Here',
#              xy=(0.8,0.002),
#              xytext=(.88,.003),
#              arrowprops=dict(arrowstyle='->', color='k')
#             )
plt.axline((1427.48041216, 0), (1427.48041216, 0.5), linewidth=2, color='r')

# create title, axis labels, and legend
plt.title('Tail pick up Data', fontsize=16)
plt.xlabel('Seconds', fontsize=14)
plt.ylabel(r'$\Delta$F/F')
plt.legend(loc='lower right', 
           bbox_to_anchor=(1.0,1.01)
          )
plt.autoscale(tight=True)
plt.show()

In [ ]:
ax3 = fig3.add_subplot(312)
p1, = ax3.plot(time, dFF, linewidth=2, color='green', label='GCaMP')
p2, = ax3.plot(tail_EVENT_x, y_scale*tail_EVENT_y+y_shift, linewidth=2, color='dodgerblue', label='Lick Bout')
ax3.set_ylabel(r'$\Delta$F/F')
ax3.set_xlabel('Seconds')
ax3.set_title('dFF with tail Bouts')
ax3.legend(handles=[p1, p2], loc='upper right')
fig3.tight_layout()
fig3

In [ ]:
tailfilepath = '/Users/sabrinay/Desktop/2022_behavior_data/20220510_fiber_photometry_ucn/20220510_fiber_photometry_data/20220510_tail_pinch_fiber_even.xlsx'
df_tail = pd.read_excel(tailfilepath)
tailfile_list = df_tail['file_name'].unique().tolist()
df_tail

In [ ]:
data_path = "/Users/sabrinay/Desktop/2022_behavior_data/20220510_fiber_photometry_ucn/20220510_fiber_photometry_data/20220510_fph_refeed_in_oft/"

In [ ]:
BLOCK_PATH = data_path + tailfile_list [0] + str("/")
fig, axs = plt.subplots(1, figsize=(15,5))
fig.suptitle("tail pinch")
data = tdt.read_block(BLOCK_PATH)
num_samples = len(data.streams._470A.data)
_470A_time = np.linspace(1, num_samples, num_samples) / data.streams._470A.fs
_405A_time = np.linspace(1, num_samples, num_samples) / data.streams._405A.fs
dff = []
for j in range (0,len(data.streams._470A.data)):
    dff.append((data.streams._470A.data[j]-data.streams._405A.data[j])/data.streams._405A.data[j])
tail_start = df_tail['Recording time'].loc[2*0]
tail_end = df_tail['Recording time'].loc[2*0 + 1]
print(tail_start)
any_t = int((tail_start)* data.streams._470A.fs) - 2
any_t_end = int((tail_end) * data.streams._470A.fs) + 2
max_any_t = max(dff[any_t: any_t_end])
min_any_t = min(dff[any_t: any_t_end])
axs.plot(_470A_time[any_t: any_t_end], dff[any_t: any_t_end], color='cornflowerblue', label="Calcium Channel")
axs.vlines(tail_start, min_any_t, max_any_t, linewidth=2, color='r')
axs.vlines(tail_end, min_any_t, max_any_t, linewidth=2, color='r')
print(max_any_t)
print(min_any_t)
plt.plot(_470A_time[any_t: any_t_end],dff[any_t: any_t_end] )
plt.show()

In [ ]:
fig, axs = plt.subplots(len(tailfile_list), figsize=(15,(len(tailfile_list) * 5)))
fig.suptitle("tail pinch")
for i in range(0, len(tailfile_list)):
    BLOCK_PATH = data_path + tailfile_list [i] + str("/")
    data = tdt.read_block(BLOCK_PATH)
    num_samples = len(data.streams._470A.data)
    _470A_time = np.linspace(1, num_samples, num_samples) / data.streams._470A.fs
    _405A_time = np.linspace(1, num_samples, num_samples) / data.streams._405A.fs
    dff = []
    for j in range (0,len(data.streams._470A.data)):
        dff.append((data.streams._470A.data[j]-data.streams._405A.data[j])/data.streams._405A.data[j])
    tail_start = df_tail['Recording time'].iloc[2*i]
    tail_end = df_tail['Recording time'].iloc[2*i + 1]
    any_t = int((tail_start)* data.streams._470A.fs) - 2
    any_t_end = int((tail_end) * data.streams._470A.fs) + 2
    max_any_t = max(dff[any_t: any_t_end])
    min_any_t = min(dff[any_t: any_t_end])
    axs[i].plot(_470A_time[any_t: any_t_end], dff[any_t: any_t_end], color='cornflowerblue', label="Calcium Channel")
    axs[i].vlines(tail_start, min_any_t, max_any_t, linewidth=2, color='r')
    axs[i].vlines(tail_end, min_any_t, max_any_t, linewidth=2, color='r')
    axs[i].legend(str([i+1]),loc="upper right")
fig.savefig( 'tail pinch' + '.png')
    

In [ ]:
# tail pinch 